In [1]:
import numpy as np
import random as rd
import math as m
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [2]:
"""Generating Test Data"""
classA = np.concatenate((np.random.randn(15, 2) * 0.3 + [2.0, 0.5], np.random.randn(15, 2) * 0.3 + [-2.0, 0.5]))
classB = np.random.randn(30, 2) * 0.3 + [0.0, 0.5]

inputs = np.concatenate((classA, classB))
targets = np.concatenate((np.ones(classA.shape[0]), -np.ones(classB.shape[0])))

N = inputs.shape[0]

permute = list(range(N))
rd.shuffle(permute)
inputs = inputs[permute, :]
targets = targets[permute]

In [3]:
"""kernel functions"""
def linearkernel(x, y):
    return np.dot(x, y)

def ploykernel(x, y):
    p = 8
    return (np.dot(x, y) + 1) ** p

def RBFkernel(x, y):
    sigma = 5
    return m.exp(-(np.linalg.norm(x - y) ** 2) / (2 * (sigma ** 2)))

kernel = linearkernel

"""objective"""
P = np.zeros((N, N))
for i in range(N):
    for j in range(N):
        P[i][j] = targets[i] * targets[j] * kernel(inputs[i], inputs[j])

def objective(alpha):
    return 0.5 * np.dot(alpha, np.dot(P, alpha)) - np.sum(alpha)

"""zerofun"""
def zerofun(alpha):
    return np.dot(alpha, targets)

"""Call minimize"""
start = np.zeros(N)

C = 100000
B = [(0, C) for b in range(N)]

XC = {'type': 'eq', 'fun': zerofun}

ret = minimize(objective, start, bounds=B, constraints=XC)
if (not ret['success']):
    raise ValueError('No solution!')
alpha = ret['x']

"""Extract the non-zero alpha values"""
nonZeroData = []
for i in range(N):
    if alpha[i] > 10 ** (-5):
        nonZeroData.append((alpha[i], inputs[i], targets[i]))
        
sv = []
for i in range(N):
    if (alpha[i] > 10 ** (-5)) & (alpha[i] < C):
        sv.append((alpha[i], inputs[i], targets[i]))
        
"""Calculate the b value"""
bvalue = 0
for data in nonZeroData:
    bvalue += data[0] * data[2] * kernel(sv[0][1], data[1])
bvalue -= sv[0][2]

"""indicator function"""
def indicator(x, y):
    ind = 0
    for data in nonZeroData:
        ind += data[0] * data[2] * kernel([x, y], data[1])
    return ind - bvalue

ValueError: No solution!

In [ ]:
"""Plotting"""
#plt.plot([p[0] for p in classA if abs(indicator(p[0], p[1]))>1+10e-5], [p[1] for p in classA if abs(indicator(p[0], p[1]))>1+10e-5], 'b.')
#plt.plot([p[0] for p in classB if abs(indicator(p[0], p[1]))>1+10e-5], [p[1] for p in classB if abs(indicator(p[0], p[1]))>1+10e-5], 'r.')
#plt.plot([p[0] for p in classA if 1-10e-5<=abs(indicator(p[0], p[1]))<=1+10e-5], [p[1] for p in classA if 1-10e-5<=abs(indicator(p[0], p[1]))<=1+10e-5], 'bx')
#plt.plot([p[0] for p in classB if 1-10e-5<=abs(indicator(p[0], p[1]))<=1+10e-5], [p[1] for p in classB if 1-10e-5<=abs(indicator(p[0], p[1]))<=1+10e-5], 'rx')

plt.plot([p[0] for p in classA], [p[1] for p in classA], 'b.')
plt.plot([p[0] for p in classB], [p[1] for p in classB], 'r.')
plt.plot([data[1][0] for data in sv], [data[1][1] for data in sv], 'yo')

xgrid = np.linspace(-5, 5)
ygrid = np.linspace(-4, 4)

grid = np.array([[indicator(x, y) for x in xgrid] for y in ygrid])

plt.contour(xgrid, ygrid, grid, (-1.0, 0.0, 1.0), colors = ('red', 'black', 'blue'), linewidths = (1, 3, 1))

plt.axis('equal')
plt.savefig('q4_2.jpg', dpi=800)
plt.show()